<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [76]:
import string
import folium
import pandas as pd
import pickle
import io
import requests
import datetime
import math
import seaborn as sns
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# get mta_stations.csv
mta_stations_csv_path = '../data/mta_stations.csv'

# Get team-wide dataframe of Brooklyn stations
foodtruck_pickle = '../pickle_jar/foodtruck_df2.pkl'

In [4]:
ft_df = pd.read_pickle(foodtruck_pickle)
ft_df.head(3)

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,HOLIDAY,ENTRIES,EXITS,LAST_ENTRIES,LAST_EXITS,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK
0,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/01/2018,09:00:00,REGULAR,Saturday,...,False,6293248,4614702,6293557,4615002,13:00:00,309,300,609,True
1,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/02/2018,09:00:00,REGULAR,Sunday,...,False,6294460,4615867,6294784,4616103,13:00:00,324,236,560,True
2,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/03/2018,09:00:00,REGULAR,Monday,...,True,6295556,4616999,6295911,4617273,13:00:00,355,274,629,True


In [5]:
mta_stations_df = pd.read_csv(mta_stations_csv_path)
mta_stations_df.head(3)

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan


In [6]:
ft_df.head(3)

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,HOLIDAY,ENTRIES,EXITS,LAST_ENTRIES,LAST_EXITS,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK
0,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/01/2018,09:00:00,REGULAR,Saturday,...,False,6293248,4614702,6293557,4615002,13:00:00,309,300,609,True
1,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/02/2018,09:00:00,REGULAR,Sunday,...,False,6294460,4615867,6294784,4616103,13:00:00,324,236,560,True
2,B004,R171,00-00-00,7 AV,BQ,7 AV BQ,09/03/2018,09:00:00,REGULAR,Monday,...,True,6295556,4616999,6295911,4617273,13:00:00,355,274,629,True


In [103]:
# dictionary to map station name to zipcode. incomplete.
# scraped using beautifulsoup of a google search

brooklyn_stations_zip_dict = {
    '7 AV BQ': ['11217'], 'PARK PLACE S': ['11238'], 'BOTANIC GARDEN S2345': ['11225'], 'PROSPECT PARK BQS': ['11226'],
    'PARKSIDE AV BQ': ['11226'], 'CHURCH AV BQ': ['11226'], 'BEVERLEY ROAD BQ': ['11226'],
    'CORTELYOU RD BQ': ['11226'], 'NEWKIRK PLAZA BQ': ['11226'], 'AVENUE H BQ': ['11230'],
    'AVENUE J BQ': ['11230'], 'AVENUE M BQ': ['11230'], 'KINGS HWY BQ': ['11229'], 'AVENUE U BQ': [],
    'NECK RD BQ': ['11229'], 'SHEEPSHEAD BAY BQ': ['11235'], 'BRIGHTON BEACH BQ': ['11235'],
    'OCEAN PKWY Q': ['11235'], 'BOROUGH HALL R2345': ['11201'], 'JAY ST-METROTEC R': [],
    'DEKALB AV BDNQR': ['11217'], 'ATL AV-BARCLAY BDNQR2345': ['11217'], 'UNION ST R': ['11215'],
    '4AV-9 ST DFGMNR': ['11231'], 'PROSPECT AV R': ['11215'], '25 ST R': ['11232'], '36 ST DNR': ['11232'],
    '45 ST R': ['11220'], '53 ST R': ['11220'], '59 ST NRW': [], 'BAY RIDGE AV R': ['11220'],
    '77 ST R': ['11209'], '86 ST R': ['11209'], 'BAY RIDGE-95 ST R': ['11209'], '8 AV N': ['11220'],
    'FT HAMILTON PKY N': [], 'NEW UTRECHT AV ND': [], '18 AV N': [], '20 AV N': ['11214'], 'BAY PKWY N': [],
    'KINGS HWY N': [], 'AVENUE U N': ['11223'], '86 ST N': ['11209'], '9 AV D': ['11219'],
    'FT HAMILTON PKY D': [], '50 ST D': [], '55 ST D': ['11219'], '71 ST D': ['11228'],
    '79 ST D': ['11220'], '18 AV D': [], '20 AV D': ['11214'], 'BAY PKWY D': ['11214'],
    '25 AV D': ['11214'], 'BAY 50 ST D': ['11214'], 'CONEY IS-STILLW DFNQ': [],
    'W 8 ST-AQUARIUM FQ': ['11224'], 'BEDFORD AV L': ['11211'], 'LORIMER ST LG': ['11211'],
    'GRAHAM AV L': ['11211'], 'GRAND ST L': ['11211'], 'MONTROSE AV L': ['11206'],
    'MORGAN AV L': ['11237'], 'JEFFERSON ST L': ['11237'], 'DEKALB AV L': ['11237'],
    'MYRTLE-WYCKOFF LM': ['11237'], 'HALSEY ST L': [], 'WILSON AV L': ['11207'],
    'BUSHWICK AV L': [], 'ATLANTIC AV L': ['11207'], 'SUTTER AV L': ['11207'],
    'LIVONIA AV L': ['11212'], 'NEW LOTS L': [], 'EAST 105 ST L': ['11236'],
    'CANARSIE-ROCKAW L': ['11236'], 'MARCY AV JMZ': ['11211'], 'HEWES ST JM': ['11211'],
    'LORIMER ST JM': [], 'FLUSHING AV JM': ['11206'], 'MYRTLE AV JMZ': [],
    'KOSCIUSZKO ST J': [], 'GATES AV JZ': ['11221'], 'HALSEY ST J': [],
    'CHAUNCEY ST JZ': ['11207'], 'ALABAMA AV J': ['11207'], 'VAN SICLEN AV JZ': [],
    'CLEVELAND ST J': [], 'NORWOOD AV JZ': ['11208'], 'CRESCENT ST JZ': ['11208'],
    'CYPRESS HILLS J': [], 'CENTRAL AV M': ['11221'], 'KNICKERBOCKER M': ['11237'],
    'HIGH ST AC': ['11201'], 'JAY ST-METROTEC ACF': ['11201'], 'HOYT-SCHER ACG': ['11201'],
    'LAFAYETTE AV C': ['11217'], 'CLINTON-WASH AV C': [], 'FRANKLIN AV ACS': ['11238'],
    'NOSTRAND AV AC': [], 'KINGSTON-THROOP C': ['11213'], 'UTICA AV AC': ['11233'],
    'RALPH AV C': ['11233'], 'ROCKAWAY AV C': ['11233'], 'BROADWAY JCT ACJLZ': [],
    'LIBERTY AV C': ['11207'], 'VAN SICLEN AVE C': ['11207'], 'SHEPHERD AV C': ['11208'],
    'EUCLID AV AC': ['11208'], 'GREENPOINT AV G': ['11222'], 'NASSAU AV G': ['11222'],
    'METROPOLITAN AV GL': [], 'BROADWAY G': ['11206'], 'FLUSHING AV G': ['11206'],
    'MYRTLE-WILLOUGH G': ['11206'], 'BEDFORD-NOSTRAN G': ['11205'], 'CLASSON AV G': ['11238'],
    'CLINTON-WASH AV G': [], 'FULTON ST G': ['11217'], 'YORK ST F': ['11201'], 'BERGEN ST FG': ['11201'],
    'CARROLL ST FG': ['11231'], 'SMITH-9 ST FG': ['11231'], '4 AV-9 ST DFGMNR': ['11215'],
    '7 AV FG': ['11215'], '15 ST-PROSPECT FG': ['11215'], 'FT HAMILTON PKY FG': [],
    'CHURCH AV FG': [], 'DITMAS AV F': ['11218'], '18 AV F': ['11230'], 'AVENUE I F': [],
    'BAY PKWY F': ['11230'], 'AVENUE N F': [], 'AVENUE P F': [], 'KINGS HWY F': ['11229'],
    'AVENUE U F': [], 'AVENUE X F': ['11223'], 'NEPTUNE AV F': ['11224'], 'CLARK ST 23': [],
    'BOROUGH HALL 2345R': ['11201'], 'HOYT ST 23': [], 'NEVINS ST 2345': ['11217'],
    'ATL AV-BARCLAY 2345BDNQR': ['11217'], 'BERGEN ST 23': [], 'GRAND ARMY PLAZ 23': [],
    'EASTN PKWY-MUSM 23': ['11238'], 'FRANKLIN AV 2345S': ['11238'], 'NOSTRAND AV 3': ['11225'],
    'KINGSTON AV 3': ['11213'], 'CROWN HTS-UTICA 34': [], 'SUTTER AV-RUTLD 3': [],
    'SARATOGA AV 3': ['11212'], 'ROCKAWAY AV 3': [], 'JUNIUS ST 3': ['11212'],
    'PENNSYLVANIA AV 3': ['11207'], 'VAN SICLEN AV 3': ['11207'], 'NEW LOTS AV 3': ['11207'],
    'PRESIDENT ST 25': ['11225'], 'STERLING ST 25': ['11225'], 'WINTHROP ST 25': ['11225'],
    'CHURCH AV 25': [], 'BEVERLY RD 25': ['11226'], 'NEWKIRK AV 25': [], 'FLATBUSH AV-B.C 25': ['11226']
}
for k,v in brooklyn_stations_zip_dict.items():
    if len(v) != 0:
        brooklyn_stations_zip_dict[k] = v[0]
    else:
        brooklyn_stations_zip_dict[k] = ''

In [104]:
brooklyn_stations_zips = pd.DataFrame.from_dict(brooklyn_stations_zip_dict, orient='index')
brooklyn_stations_zips.rename(columns = {0:'zip'}, inplace=True)

In [105]:
brooklyn_stations_zips = brooklyn_stations_zips.reset_index()
brooklyn_stations_zips.sample(5)

,index,zip
84,VAN SICLEN AV JZ,
22,UNION ST R,11215
51,BAY PKWY D,11214
73,CANARSIE-ROCKAW L,11236
100,RALPH AV C,11233


In [106]:
ft_df_zips = pd.merge(ft_df, brooklyn_stations_zips, left_on=['UNIQUE_STATION'], right_on='index')

In [107]:
ft_df_zips.sample(5)

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,EXITS,LAST_ENTRIES,LAST_EXITS,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK,index,zip
71904,N535,R220,00-00-00,CARROLL ST,FG,CARROLL ST FG,11/06/2018,07:00:00,REGULAR,Tuesday,...,19197576,5584506,19198187,11:00:00,270,611,881,True,CARROLL ST FG,11231
90149,R622,R123,00-00-04,FRANKLIN AV,2345S,FRANKLIN AV 2345S,11/06/2018,07:00:00,REGULAR,Tuesday,...,4960627,5700641,4960782,11:00:00,432,155,587,True,FRANKLIN AV 2345S,11238
4451,B020,R263,00-00-00,AVENUE H,BQ,AVENUE H BQ,09/18/2018,08:00:00,REGULAR,Tuesday,...,235634,573277,235719,12:00:00,182,85,267,True,AVENUE H BQ,11230
72252,N535,R220,00-03-01,CARROLL ST,FG,CARROLL ST FG,10/21/2018,08:00:00,REGULAR,Sunday,...,466378,3375429,466392,12:00:00,78,14,92,True,CARROLL ST FG,11231
18267,C010,R231,00-00-01,UNION ST,R,UNION ST R,10/04/2018,08:00:00,REGULAR,Thursday,...,445811,2290491,445877,12:00:00,777,66,843,True,UNION ST R,11215


Map zip code values to coordinates.

In [108]:
zip_to_coord_raw = '''ZIP     LAT    LONG
11207  40.670 -73.900
11201  40.700 -73.990
11226  40.650 -73.960
11217  40.680 -73.980
11211  40.710 -73.950
11220  40.640 -74.020
11214  40.600 -74.010
11206  40.700 -73.940
11237  40.700 -73.920
11230  40.620 -73.970
11208  40.670 -73.870
11225  40.660 -73.960
11215  40.670 -73.980
11238  40.680 -73.960
11233  40.680 -73.920
11209  40.620 -74.030
11231  40.680 -74.000
11212  40.660 -73.920
11229  40.600 -73.940
11235  40.580 -73.950
11224  40.580 -73.990
11222  40.730 -73.950
11219  40.630 -74.000
11236  40.640 -73.900
11223  40.600 -73.970
11221  40.690 -73.930
11213  40.670 -73.940
10012  40.726 -73.998
11218  40.650 -73.980
11232  40.640 -74.020
11228  40.620 -74.010
11205  40.700 -73.970
10014  40.740 -74.010
10028  40.780 -73.950'''

split = [line.split() for line in zip_to_coord_raw.split('\n')]
x = zip_to_coord_raw.split()
zip_to_geo_dict = {x[i]: str(float(x[i+1]))+'_'+str(float(x[i+2])) for i in range(3,len(x),3)}

# zip_to_coord_df = pd.DataFrame(split)
# zip_to_coord_df.columns = zip_to_coord_df.iloc[0]
# zip_to_coord_df.drop(0,inplace=True)
# zip_to_coord_df.head()

In [109]:
brooklyn_stations_zips['COORD'] = brooklyn_stations_zips['zip'][brooklyn_stations_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z])
brooklyn_stations_zips.head()

,index,zip,COORD
0,7 AV BQ,11217,40.68_-73.98
1,PARK PLACE S,11238,40.68_-73.96
2,BOTANIC GARDEN S2345,11225,40.66_-73.96
3,PROSPECT PARK BQS,11226,40.65_-73.96
4,PARKSIDE AV BQ,11226,40.65_-73.96


In [110]:
ft_df_zips['COORD'] = ft_df_zips.zip[ft_df_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z].split('_'))
ft_df_zips['LAT'] = ft_df_zips.zip[ft_df_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z].split('_')[0])
ft_df_zips['LONG'] = ft_df_zips.zip[ft_df_zips.zip != ''].apply(lambda z: zip_to_geo_dict[z].split('_')[1])

ft_df_zips.sample(4)

,C/A,UNIT,SCP,STATION,LINENAME,UNIQUE_STATION,DATE,TIME,DESC,DAY_OF_WEEK,...,LAST_TIME,DAILY_ENTRIES,DAILY_EXITS,DAILY_TOTAL,IN_BK,index,zip,COORD,LAT,LONG
61936,N124,R103,00-03-00,BROADWAY JCT,ACJLZ,BROADWAY JCT ACJLZ,10/13/2018,08:00:00,REGULAR,Saturday,...,12:00:00,403,170,573,True,BROADWAY JCT ACJLZ,,NaN,NaN,NaN
2621,B015,R098,01-03-01,CHURCH AV,BQ,CHURCH AV BQ,09/08/2018,08:00:00,REGULAR,Saturday,...,12:00:00,91,172,263,True,CHURCH AV BQ,11226,"[40.65, -73.96]",40.65,-73.96
28246,D011,R394,01-06-02,BAY PKWY,N,BAY PKWY N,09/02/2018,09:00:00,REGULAR,Sunday,...,13:00:00,728,30,758,True,BAY PKWY N,,NaN,NaN,NaN
74767,N542,R241,00-05-00,15 ST-PROSPECT,FG,15 ST-PROSPECT FG,09/18/2018,08:30:00,REGULAR,Tuesday,...,12:30:00,326,19,345,True,15 ST-PROSPECT FG,11215,"[40.67, -73.98]",40.67,-73.98


In [111]:
# from : https://towardsdatascience.com/visualizing-data-at-the-zip-code-level-with-folium-d07ac983db20

with open('../data/tl_2010_36_zcta510/nyc_zip_codes_geo.geo.json','r') as jsonFile:
    zip_geo_data = json.load(jsonFile)
tmp = zip_geo_data
    
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['ZCTA5CE10'] in list(ft_df_zips.zip.unique()):
        geozips.append(tmp['features'][i])

geozips[0].keys()

dict_keys(['type', 'properties', 'geometry'])

In [112]:
jsonFile.close()

In [113]:
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

In [114]:
open('../data/update-zip-geojson.json','w').write(
    json.dumps(new_json,  separators=(',',':'), indent=4, sort_keys=True)
)

610684

In [118]:
median_daily_entries = ft_df_zips['DAILY_TOTAL'].quantile(q=0.80)

zip_min = ft_df_zips.groupby(['LAT','LONG'])['DAILY_TOTAL'].median().min()
zip_max = ft_df_zips.groupby(['LAT','LONG'])['DAILY_TOTAL'].median().max()

# Basic map setup

folium_map = folium.Map(location=[40.650002, -73.949997],
                        zoom_start=11.5,
                        min_zoom=11,
                        max_zoom=15,
                        control_scale=True,
                        tiles='CartoDB positron')

for index, value in ft_df_zips.groupby(['zip','LAT','LONG'])['DAILY_TOTAL'].median().items():
    radius = 3 + 20*(((value - zip_min)/(zip_max - zip_min))**1.2)
    color = "Green"
    zip_mask = ft_df_zips['zip'] == index[0]
    columns = ['UNIQUE_STATION','DAILY_TOTAL']
    output = ft_df_zips[ft_df_zips['zip'] == index[0]].groupby('STATION')['DAILY_TOTAL'].median().sort_values(ascending=False).to_string().split('\n')[1:]
    html = '<strong style="color:green">Median Entries For Zip ' + str(index[0])+ ': ' + '{:,}'.format(value) + '</strong></br>'
    html += '<span>Borough-wide Entry Median: ' + '{:,}'.format(median_daily_entries) + '</span></br>'
    html += '<span>Median Household Income: $'+ '{:,}'.format(search.by_zipcode(index[0]).median_household_income) + '</span></br>'
    html += '<span>Population: '+ '{:,}'.format(search.by_zipcode(index[0]).population) + '</span></br>'
    html += '</br>Best Stations:</br>'
    html += '<span>Station Name – Median Entries</span></br>'
    for line in output[:5]:
        l = [el.strip() for el in line.split('  ')]
        cleaned = []
        for el in l:
            if len(el.strip())>0:
                cleaned += [el]
        # bold the station if it is above the borough median
        prefix = ''
        suffix = ''
        if float(cleaned[1]) > median_daily_entries:
            prefix = '<strong>'
            suffix = '</strong>'
        html += prefix +  '<div style="display:inline"><span>' + cleaned[0] + ' – ' + '{:,}'.format(float(cleaned[1])) + '</span>' + '</div>'+ suffix + '</br>'
    folium.CircleMarker(location=(index[1],index[2]),
                        radius=radius,
                        color=color,
                        popup = folium.Popup(html, max_width=300,min_width=300),
                        fill=True).add_to(folium_map)
    
for index, row in mta_stations_df.iterrows():
    radius = 0.1
    color = "blue"
    folium.CircleMarker(location=(row["GTFS Latitude"],
                                  row["GTFS Longitude"]),
                        radius=radius,
                        color=color,
                        opacity = 0.3,
                        fill_opacity = 0.3,
                        fill=True).add_to(folium_map)
    
legend_html = '''
     <div style="position: fixed; 
     bottom: 80px; left: 50px; width: 400px; height: 130px; padding:5px;
     border:2px solid grey; z-index:9999; font-size:12px;
     background-color: white;">
     <strong style='color:green'>What neighborhoods have high traffic</br>and appropriate demographics?</strong></br>
     Each Zipcode is represented by a green circle.</br>
     Blue dots are stations.</br>
     Size of the green circles represents the relative</br>
     amount of foot-traffic for all stations in that zipcode.</br>
     <strong>Bolded stations</strong> in popup are above 80th percentile in median traffic.
     </div>
     '''    

# Run a cell with the name of the map to display it in the window
folium_map.get_root().html.add_child(folium.Element(legend_html))
output_file = "Folium_Station_Zipcode_Map.html"
folium_map.save(output_file)

In [116]:
output = ft_df_zips[ft_df_zips['zip'] == '11230'].groupby('UNIQUE_STATION')['DAILY_TOTAL'].median().to_string().split('\n')[1:]

In [347]:
print(output)

['18 AV F        152.0', 'AVENUE H BQ     37.0', 'AVENUE J BQ    132.0', 'AVENUE M BQ    220.5', 'BAY PKWY F      98.0']


In [351]:
html = ''
for line in output:
    l = [el.strip() for el in line.split('   ')]
    print(l)
    cleaned = []
    for el in l:
        if len(el.strip())>0:
            cleaned += [el]
    html += '<div style="display:inline"><strong>' + cleaned[0] + '</strong>' + '<span>' + cleaned[1] + '</span>' + '</div>'

['18 AV F', '', '152.0']
['AVENUE H BQ', '37.0']
['AVENUE J BQ', '132.0']
['AVENUE M BQ', '220.5']
['BAY PKWY F', '', '98.0']


In [ ]:
html

In [335]:
zip_mask = ft_df_zips['zip'] == index[0]
columns = ['UNIQUE_STATION','DAILY_TOTAL']
output = ft_df_zips[zip_mask].groupby('UNIQUE_STATION')['DAILY_TOTAL'].median()
output

UNIQUE_STATION
36 ST DNR    553
Name: DAILY_TOTAL, dtype: int64

In [278]:
type(ft_df_zips[ft_df_zips['zip'] == '11230'][['UNIQUE_STATION','DAILY_TOTAL']].sort_values(ascending=False,by='DAILY_TOTAL')[:3])

pandas.core.frame.DataFrame

In [231]:
zip_min = ft_df_zips.groupby(['LAT','LONG'])['DAILY_TOTALS'].sum().min()
zip_max = ft_df_zips.groupby(['LAT','LONG'])['DAILY_ENTRIES'].sum().max()

# Basic map setup

m = folium.Map(location=[40.650002, -73.949997],
                        zoom_start=11.5,
                        min_zoom=11,
                        max_zoom=13,
                        control_scale=True,
                        tiles='Stamen Toner')

In [234]:
nyc_geo = r'../data/update-zip-geojson.json'

m.choropleth(
    geo_data = nyc_geo,
    fill_opacity = 0.7,
    line_opacity = 0.2,
    data = ft_df_zips,
    key_on = 'feature.properties.ZCTA5CE10',
    columns = ['zip','DAILY_ENTRIES'],
    fill_color = '#fff',
    legend_name = 'Legened'
)

# for index, value in ft_df_zips.groupby(['LAT','LONG'])['DAILY_ENTRIES'].sum().items():
#     radius = 3 + 12*(value - zip_min)/(zip_max - zip_min)
#     color = "Green"
#     folium.CircleMarker(location=(index[0],index[1]),
#                         radius=radius,
#                         color=color,
#                         fill=True).add_to(folium_map)
    
# Run a cell with the name of the map to display it in the window
m.LayerControl().add_to(m)
m

ValueError: #fff is not a valid ColorBrewer code